In [182]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import wandb
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [183]:
wandb.login(key='44fa96b263794ea519fb29399eb6b8f469eb934b')

wandb: Currently logged in as: singhbhavesh999. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [184]:
#wandb.login(key='b0bbea67b5b95cece4e781392ed3f568328da17e')

In [185]:
##44fa96b263794ea519fb29399eb6b8f469eb934b

In [186]:
train_dataset = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_train.csv',header=None)
test_dataset=pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_test.csv',header=None)
valid_dataset=pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_valid.csv' ,header=None)


In [187]:
lang2_ = 'HINDI'
SOW_token = 0 #<
EOW_token = 1 #>
UNK_token = 2 #_
PAD_token = 3 #@


In [188]:
sweep_config = {
  
  "metric": {
      "name":"validation_accuracy",
      "goal": "maximize"
  },
  "method": "bayes",
  "parameters": {
        "bidirectional": {
            "values": [True,False]
        },
        "embedding_size": {
            "values": [512, 256, 64, 32]
        },
        "hidden_layer": {
            "values": [512, 256, 128]
        },
        "epochs": {
            "values": [10,15]
        },
        "cell_type": {
            "values": ["RNN","GRU","LSTM"]
        },
        "num_layers": {
            "values": [3, 2, 1]
        },
        "dropout_encoder": {
            "values": [0.2, 0.3, 0.4]
        },
        "dropout_decoder": {
            "values": [0.2, 0.3, 0.4]
        },
        "batch_size": {
            "values": [256, 128, 64]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="DLSeqtoseq")

Create sweep with ID: e09q2jvw
Sweep URL: https://wandb.ai/singhbhavesh999/DLSeqtoseq/sweeps/e09q2jvw


In [189]:
class make_dict:
    def __init__(self,name):
        self.name=name
        self.chartoindex={}
        self.indextochar={SOW_token: "<", EOW_token: ">",UNK_token:"_",PAD_token:'@'}
        self.n_chars=4
    
    def addchar(self,word):
        for char in word:
            if char not in self.chartoindex:
                self.chartoindex[char]=self.n_chars
                self.indextochar[self.n_chars]=char
                self.n_chars+=1
            

In [190]:
# doing it on train a dataframe is given as input having input output 
# and it make the disctionary based on that and return input_dict_obj,outout_dict_obj

def Input_Output_train(df):
    #convert the dataframe to list 
    input_word=df.iloc[:,0].tolist()
    output_word=df.iloc[:,1].tolist()
    input_lang=make_dict('ENG')
    output_lang=make_dict('HINDI')
    
    # to make a dictionary of characters for the input language 
    for word in input_word:
        input_lang.addchar(word)
   # to make a dictionary of characters for teh output language 
    for word in output_word:
        output_lang.addchar(word)
    return input_lang,output_lang,input_word,output_word
  

In [191]:
#use to get max length+1 among the input words dataframe provided as input to the function 
def getlength_x(df):
    input_list = df.iloc[:,0].tolist()
    input_list_length=[]
    for word in input_list:
        input_list_length.append(len(word))
    
    input_max_length=max(input_list_length)
    return input_max_length+1


In [192]:
#use to get max length+1 among the output words dataframe provided as input to the function 
def getlength_y(df):
    output_list =df.iloc[:,1].tolist()
    output_list_length=[]
    for word in output_list:
        output_list_length.append(len(word))
    output_max_length=max(output_list_length)
    return output_max_length+1
    
    

In [193]:
input_word_dict,output_word_dict,inputword_list,outputword_list=Input_Output_train(train_dataset)
attention=True

In [194]:
attention

True

In [195]:
#use to get the data in form of list 
def getlistofwords_x(df):
    return df.iloc[:,0].tolist()

def getlistofwords_y(df):
    return df.iloc[:,1].tolist()

In [196]:
valid_input_list = getlistofwords_x(valid_dataset)
max_input_length = max(max(getlength_x(train_dataset),getlength_x(test_dataset)),getlength_x(valid_dataset))

In [197]:
#to form tensor list pass the wordlist,maxlength,dictusing 
# function that takes words as argument and return tensor 
def tensorsFormation(inputword,max_length,input_dict):
    inlist=[]
    for char in inputword:
        inlist.append(input_dict.chartoindex[char])
    inlist.append(EOW_token)
    diff=max_length-len(inlist)
    inlist.extend([PAD_token]*diff)
    wordtensor=torch.tensor(inlist, dtype=torch.long,device=device)
        
    
    return wordtensor

In [198]:
#function to create batch 
def batchcreation(data,batchsize):
    size=len(data)
    batchdata=[]
    for i in range(0,size,batchsize):
        temp=torch.stack(data[i:i+batchsize]).to(device)
        batchdata.append(temp.transpose(0,1))
            
    return batchdata

In [199]:
class EncoderRNN(nn.Module):
    def __init__(self, parameters):
        super(EncoderRNN, self).__init__()
        self.input_size=parameters['input_size_encoder']
        self.dropout = nn.Dropout(parameters['dropout_encoder'])
        self.dropout_val=parameters['dropout_encoder']
        self.num_layers = parameters['num_layers']
        self.batch_size = parameters['batch_size']
        self.embedding_size = parameters['embedding_size']
        self.hidden_size = parameters['hidden_size']
        self.cell_type = parameters['cell_type']
        self.embedding = nn.Embedding(parameters['input_size_encoder'], parameters['embedding_size'])
        self.bidirection = parameters['bidirectional']
        if(self.bidirection==True):
            self.bidirection_return_val=2
        else:
            self.bidirection_return_val=1
        
        if(self.cell_type == "GRU" ):
            self.cell_calc = nn.GRU(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val, bidirectional = self.bidirection)
        elif(self.cell_type == "LSTM"):
            self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val, bidirectional = self.bidirection)
        elif(self.cell_type == "RNN"):
            self.cell_calc = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers, dropout = self.dropout_val, bidirectional = self.bidirection)

    def forward(self, input, hidden):
        output = self.dropout(self.embedding(input).view(-1,self.batch_size,self.embedding_size))
#         output = self.dropout(embedded)
        if(self.cell_type == "LSTM"):
            output,(hidden, cell) = self.lstm(output)
        else:
            output,hidden = self.cell_calc(output, hidden)
                    
        if (self.bidirection==True):
            #rehsaping the hidden layer to calculate the avg of the two layers 
            
            dim1=hidden.size(1)
            dim2=hidden.size(2)
            hidden = hidden.reshape(2, -1, dim1, dim2)
            #print(hidden.shape)
            temp = hidden[0]
            temp.add(hidden[1])
            hidden=0.5*temp
            
            
            if(self.cell_type == "LSTM"):
                dim1= cell.size(1)
                dim2= cell.size(2)
                cell = cell.reshape(2,-1,dim1,dim2)
                temp=cell[0]
                temp.add(cell[1])
                cell=0.5*temp
            if(attention==True):
                output=output.permute(2,1,0)
                output = torch.split(output, output.shape[0]//2)
                temp1=torch.permute(output[0],(2,1,0))*0.5
                temp2=torch.permute(output[1],(2,1,0))*0.5
                output = torch.add(temp1,temp2)
        if self.cell_type == "LSTM":
            return output,hidden,cell
        else:
            return output,hidden

    def initHidden(self):
        return torch.zeros(self.bidirection_return_val*self.num_layers, self.batch_size, self.hidden_size).to(device)
#         if self.bidirection:
#             return torch.zeros(2*self.num_layers, self.batch_size, self.hidden_size).to(device)
#         else:
#             return torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device)

In [200]:
class DecoderRNN(nn.Module):
    def __init__(self, parameters):
        super(DecoderRNN, self).__init__()
        self.output_size=parameters['output_size_decoder']
        self.dropout = nn.Dropout(parameters['dropout_decoder'])
        self.dropout_val=parameters['dropout_decoder']
        self.num_layers = parameters['num_layers']
        self.batch_size = parameters['batch_size']
        self.embedding_size = parameters['embedding_size']
        self.hidden_size = parameters['hidden_size']
        self.cell_type = parameters['cell_type']
        self.embedding = nn.Embedding(parameters['output_size_decoder'], parameters['embedding_size'])
               
       
        if(self.cell_type == "GRU"):
            self.cell_calc = nn.GRU(self.embedding_size, self.hidden_size, num_layers=self.num_layers, dropout = self.dropout_val)
        elif(self.cell_type == "RNN"):
            self.cell_calc = nn.RNN(self.embedding_size, self.hidden_size, num_layers=self.num_layers, dropout = self.dropout_val)
        elif(self.cell_type == "LSTM"):
            self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, num_layers=self.num_layers, dropout = self.dropout_val)
        
        self.output_layer = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = F.relu(self.dropout(self.embedding(input).view(-1, self.batch_size, self.embedding_size)))
        if (self.cell_type=='GRU' or self.cell_type=='RNN'):
            output, hidden = self.cell_calc(output, hidden)
        else:
            output,(hidden,cell) = self.lstm(output,(hidden[0],hidden[1]))
        
        if (self.cell_type=='GRU' or self.cell_type=='RNN'):
            return self.softmax(self.output_layer(output[0])),hidden
        else:
            return self.softmax(self.output_layer(output[0])),hidden,cell
            


In [201]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, parameters):
        super(AttnDecoderRNN, self).__init__()
        self.output_size=parameters['output_size_decoder']
        self.dropout = nn.Dropout(parameters['dropout_decoder'])
        self.dropout_val=parameters['dropout_decoder']
        self.num_layers = parameters['num_layers']
        self.batch_size = parameters['batch_size']
        self.embedding_size = parameters['embedding_size']
        self.hidden_size = parameters['hidden_size']
        self.cell_type = parameters['cell_type']
        self.embedding = nn.Embedding(parameters['output_size_decoder'], parameters['embedding_size'])
        self.attention_layer = nn.Linear(self.hidden_size + self.embedding_size, max_input_length)
        self.attention_combine = nn.Linear(self.hidden_size + self.embedding_size, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_val)
        if(self.cell_type == "GRU"):
            self.cell_calc = nn.GRU(self.hidden_size, self.hidden_size, num_layers=self.num_layers, dropout = self.dropout_val)
        elif(self.cell_type == "LSTM"):
            self.lstm = nn.LSTM(self.hidden_size, self.hidden_size, num_layers=self.num_layers, dropout = self.dropout_val)
        elif(self.cell_type == "RNN"):
            self.cell_calc = nn.RNN(self.hidden_size, self.hidden_size, num_layers=self.num_layers, dropout = self.dropout_val)
        self.output_layer = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        tempencoderop=encoder_outputs.permute(1, 0, 2)
        output = self.dropout(self.embedding(input).view(-1, self.batch_size, self.embedding_size))
        temp1=output[0]
        attn_weights=None
        if self.cell_type == "LSTM":
            temp2=hidden[0][0]
            tempcat=torch.cat((temp1, temp2), 1)
            attn_weights = F.softmax(self.attention_layer(tempcat), dim=1)
        else:
            temp2=hidden[0]
            tempcat=torch.cat((temp1, temp2), 1)
            attn_weights = F.softmax(self.attention_layer(tempcat), dim=1)
            
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),tempencoderop)
        output = torch.cat((output[0], attn_applied.squeeze(1)), 1)
        output = F.relu(self.attention_combine(output).unsqueeze(0))
        if(self.cell_type == "GRU" or self.cell_type=="RNN"):
            output, hidden = self.cell_calc(output, hidden)
        else:
            output,(hidden, cell) = self.lstm(output, (hidden[0], hidden[1]))
        
        if((self.cell_type == "GRU" or self.cell_type=="RNN")):
            return self.softmax(self.output_layer(output[0])), hidden,attn_weights
        else:
            return self.softmax(self.output_layer(output[0])), hidden,cell,attn_weights
        
        



In [202]:
def train(input_batch_tensor, target_batch_tensor,  encoder_optimizer, decoder_optimizer, encoder_obj, decoder_obj,criterion,train_iter_params):
    

    decoder_optimizer.zero_grad()
    encoder_optimizer.zero_grad()
    target_length = target_batch_tensor.size(0)
    encoder_hidden = encoder_obj.initHidden()
    loss = 0
    if train_iter_params['cell_type'] == "LSTM":
        encoderop,encoder_hidden, encoder_cell = encoder_obj(input_batch_tensor, encoder_hidden)
    else:
        encoderop,encoder_hidden = encoder_obj(input_batch_tensor, encoder_hidden)
        
    decoder_input = torch.tensor([SOW_token]*train_iter_params['batch_size']).to(device)
    decoder_hidden = encoder_hidden
    if train_iter_params['cell_type'] == "LSTM":
        decoder_cell = encoder_cell
    
    teacher_forcing_ratio = 0.5

    if(random.random() < teacher_forcing_ratio):
        use_teacher_forcing = True 
    else:
        use_teacher_forcing=False
        
    if(attention==False):
        if use_teacher_forcing:
            
            for decoderinput_i in range(target_length):
                if train_iter_params['cell_type'] == "LSTM":
                    decoder_output, decoder_hidden, decoder_cell = decoder_obj(decoder_input, (decoder_hidden, decoder_cell))
                else:
                    decoder_output, decoder_hidden = decoder_obj(decoder_input, decoder_hidden)
                loss += criterion(decoder_output, target_batch_tensor[decoderinput_i])
                decoder_input = target_batch_tensor[decoderinput_i]  

        else:
            
            for decoderinput_i in range(target_length):
                if train_iter_params['cell_type'] == "LSTM":
                    decoder_output, decoder_hidden, decoder_cell = decoder_obj(decoder_input, (decoder_hidden, decoder_cell))
                else:
                    decoder_output, decoder_hidden = decoder_obj(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  

                loss += criterion(decoder_output, target_batch_tensor[decoderinput_i])
    else:
        if use_teacher_forcing:
            
            for decoderinput_i in range(target_length):
                if train_iter_params['cell_type'] == "LSTM":
                    decoder_output, decoder_hidden, decoder_cell,attention_weights = decoder_obj(decoder_input, (decoder_hidden, decoder_cell),encoderop)
                else:
                    decoder_output, decoder_hidden,attention_weights = decoder_obj(decoder_input, decoder_hidden,encoderop)
                loss += criterion(decoder_output, target_batch_tensor[decoderinput_i])
                decoder_input = target_batch_tensor[decoderinput_i]  

        else:
            
            for decoderinput_i in range(target_length):
                if train_iter_params['cell_type'] == "LSTM":
                    decoder_output, decoder_hidden, decoder_cell ,attention_weights= decoder_obj(decoder_input, (decoder_hidden, decoder_cell),encoderop)
                else:
                    decoder_output, decoder_hidden ,attention_weights= decoder_obj(decoder_input, decoder_hidden,encoderop)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  

                loss += criterion(decoder_output, target_batch_tensor[decoderinput_i])

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()
    
    loss_final= loss.item() *train_iter_params['batch_size']

    return  loss_final/ target_length

In [203]:
def train_setup(encoder, decoder, train_iter_params,parameters):
    
    
    Total_loss = 0  # Reset every print_every
    
    
    encoder_optimizer = optim.NAdam(encoder.parameters(), lr=train_iter_params['learning_rate'], weight_decay = 0.0005)
    decoder_optimizer = optim.NAdam(decoder.parameters(), lr=train_iter_params['learning_rate'], weight_decay = 0.0005)
    
    criterion = nn.CrossEntropyLoss()
    
    
    
    input_tensor_list=[]

    for i in range(len(inputword_list)):
        if(attention == False and i%train_iter_params['batch_size']==0 ):
            max_length_x=getlength_x(train_dataset[i:i+train_iter_params['batch_size']])
        elif(attention == True):
            max_length_x=max_input_length

        inputwordtensor =tensorsFormation(inputword_list[i],max_length_x,input_word_dict)
        input_tensor_list.append(inputwordtensor)
    
    output_tensor_list=[]

    for i in range(len(outputword_list)):
        if(i%train_iter_params['batch_size']==0):
            max_length_y=getlength_y(train_dataset[i:i+train_iter_params['batch_size']])
                  
        outputwordtesor = tensorsFormation(outputword_list[i],max_length_y,output_word_dict)
        output_tensor_list.append(outputwordtesor)

    valid_iptensor_list=[]

    for i in range(len(valid_input_list)):
        if(attention == False and i%train_iter_params['batch_size']==0):
            valid_max_length_x=getlength_x(valid_dataset[i:i+train_iter_params['batch_size']])
        elif(attention == True):
            valid_max_length_x=max_input_length
        validipwordtensor =tensorsFormation(valid_input_list[i],valid_max_length_x,input_word_dict)
        valid_iptensor_list.append(validipwordtensor)

    
    train_batch_input = batchcreation(input_tensor_list, train_iter_params['batch_size'])
    train_batch_output = batchcreation(output_tensor_list, train_iter_params['batch_size'])
    valid_batch_input = batchcreation(valid_iptensor_list, train_iter_params['batch_size'])


    
    
    no_of_batches=len(train_batch_output)
    no_of_datapoints=train_iter_params['batch_size']*no_of_batches
   
        
    for count in range(train_iter_params['epochs']):
        print('epoch count is ',count+1)
        for i in range(no_of_batches):
            loss = train(train_batch_input[i], train_batch_output[i], encoder_optimizer, decoder_optimizer,encoder,
                         decoder,criterion,train_iter_params)
            Total_loss += loss
                

        avg_loss = Total_loss/no_of_datapoints
        Total_loss = 0
        print("average loss at  ", count+1, "epochs is ", avg_loss)
        valid_output_list = getlistofwords_y(valid_dataset)
        valid_accuracy = cal_Accuracy(valid_batch_input, valid_output_list, train_iter_params['cell_type'], train_iter_params['batch_size'],encoder,decoder)
        
        print("Valid accuracy is ", valid_accuracy)
        #when running sweeep 
        #wandb.log({"validation_accuracy": valid_accuracy, "training_loss": avg_loss})
    


In [204]:
def evaluate(encoder_obj, decoder_obj, input_tensors, cell_type,batch_size):
    with torch.no_grad():
        
        predicted_word_tensor=[]
        encoder_hidden = encoder_obj.initHidden()
        
        if (cell_type=='GRU' or cell_type=='RNN'):
            encoder_output,encoder_hidden = encoder_obj(input_tensors, encoder_hidden)
        else:
            encoder_output,encoder_hidden, encoder_cell = encoder_obj(input_tensors, encoder_hidden)

               

        decoder_input = torch.tensor([SOW_token]*batch_size).to(device)  # SOW

        decoder_hidden = encoder_hidden
        if cell_type == "LSTM":
            decoder_cell = encoder_cell
               
            
        for i in range(input_tensors.size(0)):
            
            if(attention==False):
                if cell_type == "LSTM":
                    decoder_output, decoder_hidden, decoder_cell = decoder_obj(decoder_input, (decoder_hidden, decoder_cell))
                else:
                    decoder_output, decoder_hidden = decoder_obj(decoder_input, decoder_hidden)
                topv, topi = decoder_output.data.topk(1)
            else:
                if cell_type == "LSTM":
                    decoder_output, decoder_hidden, decoder_cell,attn_weights = decoder_obj(decoder_input, (decoder_hidden, decoder_cell),encoder_output)
                else:
                    decoder_output, decoder_hidden,attn_weights = decoder_obj(decoder_input, decoder_hidden,encoder_output)
                topv, topi = decoder_output.data.topk(1)
                
                
                                    
            temp=topi.view(1,-1).squeeze()
            predicted_word_tensor.append(temp)
            decoder_input = topi.squeeze().detach()
            
        predicted_word_tensor=torch.stack(predicted_word_tensor,dim=1).to(device)
        
        return predicted_word_tensor


In [205]:
def cal_Accuracy(input, actual_output, cell_type, batch_size,encoder1,decoder1):
    correct_output=0
    no_of_batch=len(input)
    no_of_datapoints=batch_size*no_of_batch 
    #incorrect_correct_pair=[]
    #list_word_predicted=[]
    
    for i in range(no_of_batch):
        output_tensor_predicted = evaluate(encoder1, decoder1, input[i], cell_type,batch_size)
        #print(output_tensor_predicted.shape)
                
        for j in range(output_tensor_predicted.size(0)):
            string=""
            for k in range(output_tensor_predicted.size(1)):
                target_val=output_tensor_predicted[j][k].item()
                if(target_val==EOW_token or target_val==PAD_token):
                    break
                else:
                    string =string+output_word_dict.indextochar[target_val]
            
#             list_word_predicted.append(string)
            if(actual_output[i*batch_size+j]==string):
                correct_output+=1
    
    return (correct_output/no_of_datapoints *100)
            
  
    

In [206]:
def cal_Accuracy_test(input, actual_output, cell_type, batch_size,encoder1,decoder1):
    correct_output=0
    no_of_batch=len(input)
    no_of_datapoints=batch_size*no_of_batch 
    #incorrect_correct_pair=[]
    list_word_predicted=[]
    
    for i in range(no_of_batch):
        output_tensor_predicted = evaluate(encoder1, decoder1, input[i], cell_type,batch_size)
        #print(output_tensor_predicted.shape)
                
        for j in range(output_tensor_predicted.size(0)):
            string=""
            for k in range(output_tensor_predicted.size(1)):
                target_val=output_tensor_predicted[j][k].item()
                if(target_val==EOW_token or target_val==PAD_token):
                    break
                else:
                    string =string+output_word_dict.indextochar[target_val]
            
            list_word_predicted.append(string)
            if(actual_output[i*batch_size+j]==string):
                correct_output+=1
    
    return (list_word_predicted,correct_output/no_of_datapoints *100)
            
  
    

In [207]:
def getprediceted_list_test():
    test_inputword_list=getlistofwords_x(test_dataset)
    testinput_tensor_list=[]

    for i in range(len(test_inputword_list)):
        if(attention == False and i%paramet['batch_size']==0 ):
            max_length_x=getlength_x(test_dataset[i:i+paramet['batch_size']])
        elif(attention == True):
            max_length_x=max_input_length

        testinputwordtensor =tensorsFormation(test_inputword_list[i],max_length_x,input_word_dict)
        testinput_tensor_list.append(testinputwordtensor)

    test_batch_input = batchcreation(testinput_tensor_list, paramet['batch_size'])
    test_output_list= getlistofwords_y(test_dataset)
    list_word_predicted,test_accuracy = cal_Accuracy_test(test_batch_input, test_output_list, paramet['cell_type'], paramet['batch_size'],encodertest,decodertest)
    print('test_accuracy is ',test_accuracy)
    return list_word_predicted

In [208]:
#to get the predicted list first with vanilla version then with attention then comapre what attention 
#predicted correct 
attention = True
paramet= {

"input_size_encoder":input_word_dict.n_chars,
"output_size_decoder":output_word_dict.n_chars,
"hidden_size":  512,
"num_layers":  1,
"dropout_encoder":0.4,
"dropout_decoder": 0.2,
"batch_size": 128,
"embedding_size": 512,
"epochs": 15,
"cell_type":"GRU",
"bidirectional": True
    
}

encodertest = EncoderRNN(paramet).to(device)
if(attention==False):
    decodertest = DecoderRNN(paramet).to(device)
else:
    decodertest=AttnDecoderRNN(paramet).to(device)
    

train_iter_params={
    'epochs':paramet['epochs'],
    'learning_rate':0.001,
    'batch_size':paramet['batch_size'],
    'cell_type':paramet['cell_type']  
    
    
 }
train_setup(encodertest, decodertest, train_iter_params,paramet)
list_word_predicted_atten=getprediceted_list_test()

test_output_df = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_test.csv',names=['input','output'])
test_output_df['predicted_output']=list_word_predicted_atten
test_output_df.to_csv('resultattention.csv')




attention = False

paramet= {

"input_size_encoder":input_word_dict.n_chars,
"output_size_decoder":output_word_dict.n_chars,
"hidden_size":  512,
"num_layers":  2,
"dropout_encoder":0.4,
"dropout_decoder": 0.3,
"batch_size": 64,
"embedding_size": 512,
"epochs": 10,
"cell_type":"LSTM",
"bidirectional": True
    
}

encodertest = EncoderRNN(paramet).to(device)
if(attention==False):
    decodertest = DecoderRNN(paramet).to(device)
else:
    decodertest=AttnDecoderRNN(paramet).to(device)
    

train_iter_params={
    'epochs':paramet['epochs'],
    'learning_rate':0.001,
    'batch_size':paramet['batch_size'],
    'cell_type':paramet['cell_type']  
    
    
 }
train_setup(encodertest, decodertest, train_iter_params,paramet)
list_word_predicted_vanilla=getprediceted_list_test()

test_output_df = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_test.csv',names=['input','output'])
test_output_df['predicted_output']=list_word_predicted_vanilla
test_output_df.to_csv('resultvanilla.csv')

    
test_actual_output=getlistofwords_y(test_dataset)
count_corrected_pre=0
for i in range(len(test_actual_output)):
    if(test_actual_output[i]==list_word_predicted_atten[i] and test_actual_output[i]!=list_word_predicted_vanilla[i]):
        print('actual output is ',test_actual_output[i])
        print('converted output using attention ',list_word_predicted_atten[i])
        print('converted output without attention ',list_word_predicted_vanilla[i])
        print('-------------------------------------------------------------------')
        count_corrected_pre+=1
        if(count_corrected_pre==5):
            break



/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


epoch count is  1
average loss at   1 epochs is  1.1789383610978772
Valid accuracy is  23.8525390625
epoch count is  2
average loss at   2 epochs is  0.5418024903719869
Valid accuracy is  31.298828125
epoch count is  3
average loss at   3 epochs is  0.4400207104249904
Valid accuracy is  33.642578125
epoch count is  4
average loss at   4 epochs is  0.4100514598396364
Valid accuracy is  34.7412109375
epoch count is  5
average loss at   5 epochs is  0.40087428461479774
Valid accuracy is  36.0107421875
epoch count is  6
average loss at   6 epochs is  0.3837528674047855
Valid accuracy is  36.3037109375
epoch count is  7
average loss at   7 epochs is  0.35930055634363706
Valid accuracy is  37.841796875
epoch count is  8
average loss at   8 epochs is  0.33032367202371254
Valid accuracy is  38.232421875
epoch count is  9
average loss at   9 epochs is  0.3192617187894774
Valid accuracy is  39.4287109375
epoch count is  10
average loss at   10 epochs is  0.3173701284734204
Valid accuracy is  39.

In [209]:
# paramet= {

# "input_size_encoder":input_word_dict.n_chars,
# "output_size_decoder":output_word_dict.n_chars,
# "hidden_size":  512,
# "num_layers":  2,
# "dropout_encoder":0.4,
# "dropout_decoder": 0.3,
# "batch_size": 64,
# "embedding_size": 512,
# "epochs": 10,
# "cell_type":"LSTM",
# "bidirectional": True
    
# }

In [173]:
# encodertest = EncoderRNN(paramet).to(device)
# if(attention==False):
#     decodertest = DecoderRNN(paramet).to(device)
# else:
#     decodertest=AttnDecoderRNN(paramet).to(device)
    

# train_iter_params={
#     'epochs':paramet['epochs'],
#     'learning_rate':0.001,
#     'batch_size':paramet['batch_size'],
#     'cell_type':paramet['cell_type']  
    
    
#  }
# train_setup(encodertest, decodertest, train_iter_params,paramet)
    

epoch count is  1
average loss at   1 epochs is  1.00620045134134
Valid accuracy is  25.78125
epoch count is  2
average loss at   2 epochs is  0.5024777852100586
Valid accuracy is  31.7626953125
epoch count is  3
average loss at   3 epochs is  0.43877922390554125
Valid accuracy is  34.1796875
epoch count is  4
average loss at   4 epochs is  0.38649523156762283
Valid accuracy is  36.03515625
epoch count is  5
average loss at   5 epochs is  0.37700234202074023
Valid accuracy is  37.4267578125
epoch count is  6
average loss at   6 epochs is  0.36325877447474453
Valid accuracy is  37.451171875
epoch count is  7
average loss at   7 epochs is  0.34135886038117086
Valid accuracy is  37.8662109375
epoch count is  8
average loss at   8 epochs is  0.3397493875315884
Valid accuracy is  38.1591796875
epoch count is  9
average loss at   9 epochs is  0.3268018930633557
Valid accuracy is  37.255859375
epoch count is  10
average loss at   10 epochs is  0.3179253804590276
Valid accuracy is  36.64550781

In [153]:
# #to see on test dataset 
# test_inputword_list=getlistofwords_x(test_dataset)
# testinput_tensor_list=[]

# for i in range(len(test_inputword_list)):
#     if(attention == False and i%paramet['batch_size']==0 ):
#         max_length_x=getlength_x(test_dataset[i:i+paramet['batch_size']])
#     elif(attention == True):
#         max_length_x=max_input_length

#     testinputwordtensor =tensorsFormation(test_inputword_list[i],max_length_x,input_word_dict)
#     testinput_tensor_list.append(testinputwordtensor)

# test_batch_input = batchcreation(testinput_tensor_list, paramet['batch_size'])
# test_output_list= getlistofwords_y(test_dataset)
# list_word_predicted,test_accuracy = cal_Accuracy_test(test_batch_input, test_output_list, paramet['cell_type'], paramet['batch_size'],encodertest,decodertest)
# print(test_accuracy)
# #39.1357421875

31.7138671875


In [157]:
# list_word_predicted_vanilla=getprediceted_list_test()


In [159]:
# # #to form csv file of the predicted letters 
# test_output_df = pd.read_csv('/kaggle/input/aksharantar/aksharantar_sampled/hin/hin_test.csv',names=['input','output'])
# test_output_df['predicted_output']=list_word_predicted_vanilla
# test_output_df.to_csv('resultvanilla2.csv')



In [ ]:
# to get the words which are corrected by attention 

In [102]:
# def runsweep():
#     config = None
#     with wandb.init(config = config, entity = 'singhbhavesh999') as run:
#         config = wandb.config
#         run.name='hl_'+str(config.hidden_layer)+'_bs_'+str(config.batch_size)+'_ct_'+config.cell_type
#         #run.name = "EBS_{}_NL_{}_HL_{}_CellT_{}_BS_{}".format(config.embedding_size, config.num_layers, config.hidden_layer, config.cell_type,config.batch_size)
#         param = {

#             "input_size_encoder":input_word_dict.n_chars,
#             "output_size_decoder":output_word_dict.n_chars,
#             "hidden_size":  wandb.config.hidden_layer,
#             "num_layers":  wandb.config.num_layers,
#             "dropout_encoder":wandb.config.dropout_encoder,
#             "dropout_decoder": wandb.config.dropout_decoder,
#             "batch_size": wandb.config.batch_size,
#             "embedding_size": wandb.config.embedding_size,
#             "epochs": wandb.config.epochs,
#             "cell_type":wandb.config.cell_type,
#             "bidirectional": wandb.config.bidirectional

#         }
        
        
        

       
#         encoder1 = EncoderRNN(param).to(device)
#         if(attention==False):
#             decoder1 = DecoderRNN(param).to(device)
#         else:
#             decoder1=AttnDecoderRNN(param).to(device)


#         train_iter_params={
#             'epochs':param['epochs'],
#             'learning_rate':0.001,
#             'batch_size':param['batch_size'],
#             'cell_type':param['cell_type']  
#             }

#         train_setup(encoder1, decoder1, train_iter_params,param)

 

# wandb.agent(sweep_id, runsweep, count = 2)

wandb: Agent Starting Run: 6ziz1j0b with config:
wandb: 	batch_size: 64
wandb: 	bidirectional: True
wandb: 	cell_type: LSTM
wandb: 	dropout_decoder: 0.3
wandb: 	dropout_encoder: 0.2
wandb: 	embedding_size: 64
wandb: 	epochs: 15
wandb: 	hidden_layer: 256
wandb: 	num_layers: 2
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


epoch count is  1
average loss at   1 epochs is  1.225223551959046
Valid accuracy is  11.9384765625
epoch count is  2
average loss at   2 epochs is  0.7057481039882307
Valid accuracy is  20.6787109375
epoch count is  3
average loss at   3 epochs is  0.6063149021120399
Valid accuracy is  23.779296875
epoch count is  4
average loss at   4 epochs is  0.5509449524803557
Valid accuracy is  26.0009765625
epoch count is  5
average loss at   5 epochs is  0.5083218414774848
Valid accuracy is  28.271484375
epoch count is  6
average loss at   6 epochs is  0.4741551306824455
Valid accuracy is  30.5908203125
epoch count is  7
average loss at   7 epochs is  0.4692886567389637
Valid accuracy is  31.591796875
epoch count is  8
average loss at   8 epochs is  0.44553230925288867
Valid accuracy is  32.2509765625
epoch count is  9
average loss at   9 epochs is  0.43833090340021835
Valid accuracy is  32.2998046875
epoch count is  10
average loss at   10 epochs is  0.420538055465469
Valid accuracy is  33.76

training_loss,█▄▃▂▂▂▂▂▂▁▁▁▁▁▁
validation_accuracy,▁▄▅▅▆▇▇▇▇█▇████
training_loss,0.37144
validation_accuracy,33.95996


In [ ]:
# wandb.agent('a0km7f3z', runSweep, count = 2)